In [3]:
%matplotlib inline                                    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import itertools
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.optimize import leastsq
import matplotlib.gridspec as gridspec
from dateutil.parser import parse

In [4]:
def func_treatment(data, head, year, col):

    pd.options.mode.chained_assignment = None                       # permit to avoid error in copying 
    
    cols = col 
    for c in cols:
        data[c] = pd.to_numeric(data[c], errors='coerce')           # cast the type str to numeric
        
    # Select a part of the dataframe where the year is > at year passed in the function 
    data = data[(data[head[3]].astype(int)<=datetime.now().year) & (data[head[3]].astype(int)>=year) ]
    data = data[data[head[4]]>0]
    date = [datetime(data[head[3]].iat[i],data[head[4]].iat[i],1) for i in range(len(data))]
    data['Date'] = date                                             # .astype() permit to cast a type to another     
                                                                    # passed in the () 
    # ---- Create the age to compare --------------------------------    
    #print(data.data_month[:5])
    if(len(data)>0):
        test=data.data_month.iat[0].split('_')
        test2=data.data_month.iat[0].split('/')
        if(len(test)>1):
            data_month = [i.split('_') for i in data.data_month.values]
            data_month = [datetime(int(i[0]),int(i[1]),1) for i in data_month]          # make a datetime
        else:
            if(len(test2)):
                data_month = [i.split('/') for i in data.data_month.values]
                data_month = [datetime(int(i[2]),int(i[1]),int(i[0])) for i in data_month] 
            #data_month = [i.split('-') for i in data.data_month.values]
            else:
                data_month = parse(data.data_month)
        #print('third',data_month[:5])
    #print (data_month[:5])
    else:
        data_month = []
    
    date_age=[round((data_month[i]-date[i]).days/30) for i in range(len(date))] # compute the age in months
    
    data['Age_months']=date_age                                                 # save in a new column
    data = data[data.Age_months>=0]                                             # keep all positive or null age 
    # ---- Data cleaning --------------------------------------------
    a = data[head[0]].fillna(' ')
    data[head[0]] = a.values
    a = data[head[1]].apply(lambda x : x if x>0 else 0 )
    data[head[1]] = a.values    
    return data

def func_outliers(data):                                       # remove outliers from data
    data=data[data.Prix<100000]
    data=data[data.Prix>500]
    years = data[head[3]].unique()                             # list of years unique
    years = list(years)
    years.sort()                                               # sort the list ascending
    t = []
    if(len(years)<3):                                          # if we have 1 or 2 years we take all the data
        return data
    for i in years :
        df = data[data[head[3]]==i]                            # select a subdataframe by year
        months = df[head[4]].unique()                          # return a list of month
        for j in months:
            df2 = df[df[head[4]]==j]                           # select a subsubdataframe by month in the year
            mean = df2[head[1]].median(axis=0)                 # compute the median of the price for the month
            std  = df2[head[1]].std(axis=0)                    # compute the standard deviation
            df_fin = df2[(df2[head[1]] < (mean+ 3* std)) ]     # select price in the interval median+/-2std
            df_fin = df_fin[(df_fin[head[1]]>(mean-3*std))]
            t.append(df_fin)                                   # create a list of subsubdataframe
        d = pd.concat(t, axis=0)                               # create a final dataframe of the points without 
                                                               # outliers
    d = d.drop_duplicates(d[['Prix','KM','Date']],keep='last')
    return d

def func_mean_median(data, name):
    # ---- Mean Median computation --------------------------
    mean     = []                                # Empty list to stock the results of the computation
    median   = []
    err      = []
    km_value = []
    months   = []
    for i in data[name].unique():
        d  = data[data[name]==i]                # Select a subdataframe by date
        z  = d[head[1]].mean(axis=0)            # Compute the mean of the price
        z4 = d[head[1]].std(axis=0)             # Compute the standard deviation of the price 
        z2 = d[head[1]].median(axis=0)          # Compute the median of the price
        z3 = d[head[2]].mean(axis=0)            # Compute the mean of the kilometer
        #months.append(d.Age_months.mean(0))    # Save the corresponding month
        mean.append(z)                          # Save the values in the lists 
        median.append(z2)
        err.append(z4)
        km_value.append(z3)
    # ---- Save the results in a dictionnary --------------------------------------------------------------------------
    dict_values = { name       :pd.Series(np.array(df_select[name].unique())),
                   'Mean'      :pd.Series(np.array(mean)),
                   'Median'    :pd.Series(np.array(median)),
                   'Error'     :pd.Series(np.array(err)),
                   'km'        :pd.Series(np.array(km_value))}#,
                   #'Age_months':pd.Series(np.array(months))}
    # Pass from dict to dataframe by specifying the name of the columns - correspond to the columns' order
    df_results = pd.DataFrame(columns = [name, 'Mean','Median','Error','km'],data=dict_values)
    return df_results

def func_select(df,mmpl):
    #print(list(df))
    df_carac = df[['Modele', 'Prix', 'KM', 'Annee', 'Mois','Carrosserie', \
               'Carburant', 'Litre', 'Puissance', '4RM',  'data_month']]    
    cols = ['Prix', 'KM', 'Annee', 'Mois', 'Puissance', 'Litre']# isolate columns needed to be cast
    head = list(df_carac)                                       # save the header in list 
    #print(df_carac)
    #df_carac = df_carac.dropna(subset=cols)                     # drop rows where NaN values 
    #df_model = df_carac[df_carac[head[0]]==model]               # select a subdataframe 
    #print(mmpl)
    #test = df_carac[df_carac['Modele']==mmpl]#[0]]
    #print(len(df_carac), len(test),mmpl)
    df_carac = df_carac[df_carac['Mois'].astype(int)>0]         # keep rows with month > 0 
    df_carac = df_carac[df_carac['Mois'].astype(int)<=12]       # keep rows with month < 13
    #df_carac = df_carac[df_carac['Litre'].astype(float)>0]      # keep rows with liter > 0 
    #df_carac = df_carac.dropna(subset=['Litre','Puissance'])    # drop rows where liter and power are NaN values 
    #df_carac = df_carac[df_carac['Puissance'].astype(float)>0]  # keep rows where power > 0 
    df_carac = df_carac.sort_values(by='data_month')            # sort the dataframe by the data_month (extraction)
   
    df_model = df_carac[(df_carac['Modele']==mmpl[0]) & (df_carac['Puissance']==mmpl[1])\
               & (df_carac['Litre'].astype(float)==float(mmpl[2]))]
    #print(len(df_model))
    '''if(len(df_model)<20):
        df_model = df_carac[(df_carac['Modele']==mmpl[0])&(df_carac['Puissance']==mmpl[1])]
                            
        #print(df_model['Puissance'][:2])
    if (len(df_model)<20):
        df_model = df_carac[(df_carac['Modele']==mmpl[0])&\
                                (df_carac['Litre'].astype(float)==float(mmpl[2]))]
        #print(df_model['Litre'][:2])
    if (len(df_model)<20):
        df_model = df_carac[df_carac['Modele']==mmpl[0]]
        #print(df_model['Modele'][:2])
    '''  
    #print(len(df_model))
    return df_model, cols                                       # return the select dataframe and a columns' list

# ---- Functions for the different fits -------------------------------------------------------------------------------
def func_fitexp(x, y):                                      # fit exponential with numpy package   
    res = np.polyfit(x, np.log(y), 1,w=np.sqrt(y))
    res_tot = np.exp(res[1])*np.exp(res[0]*x)
    return res_tot, res

def func_exp(x, y):                                         # fit exponential with scipy package 
    res_fit = curve_fit(lambda t,a,b: a*np.exp(b*t),  x,  y,  p0=(20000, 1e-6))
    #print(res_fit[0])
    return res_fit[0][0] * np.exp(res_fit[0][1] * x)

def func_exp2(p, x, z):                                     # exponential function with multiple parameters  
#    a, b, t, t2 = p[0], p[1], p[2], p[3]
    a, b, c, d, t, t2 = p[0], p[1], p[2], p[3], p[4], p[5]
#    return a*np.exp(b + x*t + z*t2)
    return a*np.exp(b + pow(x,c)*t + pow(z,d)*t2)

def err(p, x,z, y):                                         # error function compare the previous to the estimate to
    return func_exp2(p, x,z) - y                            # minimise the residuals 

def func_poly(p, x, z):                                     # polynomial function with multiple parameters
    a, t, t2, t3, t4, t5, t6 = p[0], p[1], p[2], p[3], p[4], p[5], p[6]
    return a + x*t+ z*t2 + pow(x,2)*t3 + pow(z,2)*t4 + pow(x,3)*t5 + pow(z,3)*t6

def err_poly(p, x,z, y):                                    # error function compare the previous to the estimate to
    return func_poly(p, x,z) - y                            # minimise the residuals 

def func_annual_devalo3(y):
    devalo = y / y.shift(+1)
    #print(y[:10])
    #print(devalo[:10])
    devalo_mean= []
    x_mean = []
    i = 0
    #annee_max = np.arange(round(data.Age_months.max()/12))
    #print(annee_max)
    while i < len(y) :
        devalo_mean.append(100-100*pow(devalo[i:i+12].mean(), 12))
        i+=12
        x_mean.append(i)
    return devalo_mean, x_mean

In [9]:
def func_mmcclbp2(test, mmcclbp):
    try:
        test = df[ (df['Modele']==mmcclbp[0]) &(df['Carrosserie']==mmcclbp[1]) &\
          (df['Carburant']==mmcclbp[2]) & (df['Litre']==float(mmcclbp[3])) &(df['Puissance']==mmcclbp[4]) ]
    except:
        try:
            test = df[ (df['Modele']==mmcclbp[0]) &(df['Carrosserie']==mmcclbp[1]) &\
          (df['Carburant']==mmcclbp[2]) & (df['Litre']==float(mmcclbp[3]))]
        except:
            try:
                test = df[(df['Modele']==mmcclbp[0]) &(df['Carrosserie']==mmcclbp[1]) &\
          (df['Carburant']==mmcclbp[2])]
            except:
                try:
                    test = df[ (df['Modele']==mmcclbp[0]) &(df['Carrosserie']==mmcclbp[1])]
                except:
                    try:
                        test = df[ (df['Modele']==mmcclbp[0])]
                    except:
                        print('Empty data frame')
    return test

In [6]:
name = ['VIN','Cote_D', 'VR' ,'Age','KM','Marque', 'Modele', 'Carburant', 'Carrosserie','Litre','Boite','Puissance','4RM','Version_id',\
        'Porte','MCCLBP','Annee','Mois','Jour','KMM','KM_VR','data_month','Date_VR','Sur_KM']
df_test = pd.read_csv('../test_vr_20170512.csv',sep=';',dtype=str,header=0,names=name,index_col=False)
df_test['Date_VR'] = df_test['Date_VR'].apply(lambda x: x.split()[0])

print(df_test.head())

                 VIN Cote_D     VR Age      KM   Marque     Modele Carburant  \
0  vf30arhe8ds049762  15269  13411  35   64000  PEUGEOT       5008         2   
1  vf3xuahz8cz049873      0      0  41   94300  PEUGEOT  EXPERT VU         2   
2  vf3xs9hm0cz048011  10114   8931  41  115300  PEUGEOT  EXPERT VU         2   
3  vf3cc9hpkdt177192  12489  10897  27   31600  PEUGEOT        208         2   
4  vf3cu9hp0ey011371  15939  14167  26   40900  PEUGEOT       2008         2   

  Carrosserie Litre  ...   Porte                          MCCLBP Annee Mois  \
0          13  2.00  ...       5  VO-82-2428-2-13-1-2.00-150---N  2013    4   
1          22  2.00  ...       4  VO-82-1176-2-22-1-2.00-125---N  2012   10   
2          22  1.60  ...       0   VO-82-1176-2-22-1-1.60-90---N  2012   10   
3           2  1.60  ...       5    VO-82-3133-2-2-1-1.60-92---N  2013   12   
4           5  1.60  ...       5    VO-82-3239-2-5-1-1.60-92---N  2014    1   

  Jour   KMM   KM_VR           data_month   

In [7]:
name = ['data_month','Marque','Modele','Carrosserie','Carburant','Litre','Puissance','Boite','4RM','Prix','KM','Annee'\
        ,'Mois','Porte','doublon_id_2','site']
df_lacentrale = pd.read_csv('../Docs/Files_fr/Etude_Christophe/car_FR_PSA_lacentrale.csv.gz', sep=';',dtype=str ,\
                 compression = 'gzip',header=0, names=name)
print(df_lacentrale.head())

   data_month   Marque            Modele Carrosserie Carburant Litre  \
0  15/02/2016  CITROEN  C4 GRAND PICASSO   MONOSPACE    DIESEL   1.6   
1  15/02/2016  CITROEN  C4 GRAND PICASSO   MONOSPACE    DIESEL   1.6   
2  15/02/2016  CITROEN               DS3     BERLINE    DIESEL   1.6   
3  15/02/2016  CITROEN               DS5     BERLINE    DIESEL   2.0   
4  15/02/2016  CITROEN          BERLINGO   MONOCORPS    DIESEL   1.6   

  Puissance        Boite 4RM   Prix      KM Annee Mois Porte doublon_id_2  \
0       112  AUTOMATIQUE   N  10990  106786  2011    5     5       100063   
1       109     MANUELLE   N  12390   92760  2011    8     5       100081   
2        92     MANUELLE   N  10990  106035  2011   11     3       100104   
3       163     MANUELLE   N  23990   43288  2014    5     5       101215   
4        92     MANUELLE   N  16990    5906  2015    5     5       101217   

         site  
0  lacentrale  
1  lacentrale  
2  lacentrale  
3  lacentrale  
4  lacentrale  


In [8]:
#name2 = ['data_month','Marque','Modele','Carrosserie','Carburant','Litre','Puissance','Boite','4RM','Prix','KM','Annee'\
#        ,'Mois','Porte','doublon_id_2','site']
name2 = ['data_month', 'doublon_id_2', 'Marque', 'Modele', 'Nom', 'Carburant', 'Carrosserie', 'Annee', 'Mois', 'Age',\
         'KM', 'Prix', 'Cote_VO', 'Ecart_prix_cote', 'site', '4RM', 'Porte', 'Couleur', 'Litre', 'Puissance']
#brand = 'PSA'
#folder= '../Docs/Scoring/Scoring_'+brand+'/' 
#name  = folder+'Scoring_'+brand+'_2016.csv'
name  = '../PSA_histo3.csv'
df_2017 = pd.read_csv(name+'.gz', sep=';',dtype=str ,\
                 compression = 'gzip',header=0, names=name2)
print(list(df_2017))
print(df_2017.head())

['data_month', 'doublon_id_2', 'Marque', 'Modele', 'Nom', 'Carburant', 'Carrosserie', 'Annee', 'Mois', 'Age', 'KM', 'Prix', 'Cote_VO', 'Ecart_prix_cote', 'site', '4RM', 'Porte', 'Couleur', 'Litre', 'Puissance']
  data_month doublon_id_2   Marque      Modele               Nom Carburant  \
0    2010_01         3353  CITROEN  C4 PICASSO  118d Excellis 3p    DIESEL   
1    2010_01         5418  CITROEN         2CV               NaN   ESSENCE   
2    2010_01         5419  CITROEN         2CV               NaN   ESSENCE   
3    2010_01         5423  CITROEN          ZX               NaN    DIESEL   
4    2010_01         5425  CITROEN         NaN               NaN   ESSENCE   

  Carrosserie  Annee Mois  Age      KM   Prix Cote_VO Ecart_prix_cote site  \
0   MONOSPACE  20009    5    0   10500  24800       0             0.0   1    
1     BERLINE   1989    3  251  108150   5900       0             0.0   1    
2     BERLINE   1985    3  299   38900   5900       0             0.0   1    
3     BE

In [6]:
test_car = df_lacentrale[(df_lacentrale['Modele']=='DS5') &(df_lacentrale['Puissance']=='120')]#\
#               & (df_lacentrale['Litre']=='1.6')]
print(list(test_car))
mmpl_LC = df_lacentrale[['Modele','Puissance','Litre']]
mmpl_LC = mmpl_LC.drop_duplicates()
mmpl_LC = mmpl_LC.dropna()
#print(mmpl_LC[mmpl_LC['Modele']=='DS5'])
test_car = test_car[(test_car.Puissance.astype(int) > 0) & (test_car.Litre.astype(float)>0)]
print(test_car.Litre.unique())
print(len(test_car))
#print(test_car[['Prix','Puissance', 'Litre']])

['data_month', 'Marque', 'Modele', 'Carrosserie', 'Carburant', 'Litre', 'Puissance', 'Boite', '4RM', 'Prix', 'KM', 'Annee', 'Mois', 'Porte', 'doublon_id_2', 'site']
['1.6']
745


In [29]:
test_car2 = df_2017[(df_2017['Modele']=='DS5') &(df_2017['Puissance']=='120')]#\
#               & (df_2017['Litre']=='2.0')]
df_mcclbp_2017 = df_2017[['Modele','Carrosserie','Carburant','Litre','Puissance']]
df_mcclbp_2017 = df_mcclbp_2017.drop_duplicates()
df_mcclbp_2017 = df_mcclbp_2017.dropna()
#print(mmpl_2017[mmpl_2017['Modele']=='DS5'])
print(df_mcclbp_2017[:10])
test_car2 = test_car2[(test_car2.Puissance.astype(int) > 0) & (test_car2.Litre.astype(float)>0)]
print(test_car2.Litre.unique())
print(len(test_car2))
#print(test_car2[['Prix','Puissance', 'Litre']])

           Modele Carrosserie Carburant Litre Puissance
0      C4 PICASSO   MONOSPACE    DIESEL   2.0       0.0
1             2CV     BERLINE   ESSENCE   0.0       0.0
3              ZX     BERLINE    DIESEL   1.9       0.0
5   XSARA PICASSO   MONOSPACE    DIESEL   1.6       0.0
7              AX     BERLINE   ESSENCE   1.0       0.0
9              AX     BERLINE   ESSENCE   0.0       0.0
11             AX     BERLINE    DIESEL   0.0       0.0
12             AX  UTILITAIRE   ESSENCE   0.0       0.0
14             AX     BERLINE   ESSENCE   1.1       0.0
16    BERLINGO VU     SOCIETE    DIESEL   1.9       0.0
[]
0


In [28]:
df_mcclbp = df_test[['Modele','Carrosserie','Carburant','Litre','Puissance']]
df_mcclbp = df_mcclbp.drop_duplicates()
print(df_mcclbp[:10])
df_mcclbp = df_mcclbp.dropna()
print(df_mcclbp[df_mcclbp['Modele']=='DS5'])
print(len(df_mcclbp))

       Modele Carrosserie Carburant
0        5008          13         2
1   EXPERT VU          22         2
3         208           2         2
4        2008           5         2
6         508           4         2
8         308           4         2
9         308           2         2
11       3008          13         2
12        208          22         2
14       3008          13        24
     Modele Carrosserie Carburant
852     DS5           2         2
1214    DS5           2        24
1478    DS5           2         1
66


In [24]:
a = df_mcclbp.iloc[0]
print(a.values)
b = func_mmcclbp2(df_2017, a.values)

['5008' '13' '2' '2.00' '150']
Empty data frame


In [9]:
year_min = 2000  

In [10]:
mpl = mmpl[0+1:0+2]
print(mpl)
print(mpl['Modele'].iat[0])

      Modele Puissance Litre
1  EXPERT VU       125  2.00
EXPERT VU


In [20]:
models= pd.concat([df_test.Modele, df_test.Puissance, df_test.Litre], axis=1)
print(models[:10])
print(len(models))
models=models.drop_duplicates()
print(models[130:])
print(len(models))
#list(models[:1])

      Modele Puissance Litre
0       5008       150  2.00
1  EXPERT VU       125  2.00
2  EXPERT VU        90  1.60
3        208        92  1.60
4       2008        92  1.60
5        208        92  1.60
6        508       115  1.60
7        508       115  1.60
8        308       115  1.60
9        308       120  1.60
2123
                Modele Puissance Litre
1456        C4 PICASSO       150  2.00
1467               DS3       110  1.20
1478               DS5       165  1.60
1480         C4 CACTUS        92  1.60
1485               DS5       210  1.60
1509        C4 PICASSO       120  1.60
1530                C3        70   NaN
1544  C4 GRAND PICASSO        90  1.60
1574               DS3        90  1.60
1635           NEMO VU        70  1.40
1727         JUMPER VU       NaN   NaN
1848          BERLINGO       100  1.60
1883         JUMPER VU       150  2.20
1889          BERLINGO        75  1.60
1952            C-ZERO        67  0.00
1995               DS4       180  2.00
2014         

In [21]:
#mmpl = [mmpl[1:2]]
t = []
#models= zip(df_test.Modele, df_test.Puissance).unique() 
#print(models)
for i in range(len(models)-1):#range(len(mmpl)-1):
    print(models[i:i+1])
    nb_cars = 0 
    mpl = [models[i:i+1]['Modele'].iat[0],models[i:i+1]['Puissance'].iat[0],models[i:i+1]['Litre'].iat[0] ] #[mmpl[i+1:i+2]['Modele'].iat[0]]#, mmpl[i+1:i+2]['Puissance'].iat[0], mmpl[i+1:i+2]['Litre'].iat[0]] 
    #print(mpl)
    df_model, cols = func_select(df_2017, mpl)
    
    if(len(df_model)>1):
        head = list(df_model)
        df_model = func_treatment(df_model,head,year_min, cols)             # make the treatment of the subdataframe
        if(len(df_model)>100):
            df_select= func_outliers(df_model)                                  # reduce the parameters space
            #print(len(df_select))
        else:
            df_select = df_model
        nb_cars += len(df_select)                                           # save the cars' number 
        df_results = func_mean_median(df_select, 'Age_months')              # compute the mean median per age
        df_results = df_results.sort_values(by='Age_months')                # sort the dataframe 
    
    # ---- Depreciation ----------------------------------------------------------------------------------------------
    # ---- Polynomial fit
    
        p1=[20000,-1e-3,1,1,1,-1e-2, -1e-6]
        df = df_select
        #print(df_select.head())
        param_poly, ier = leastsq(err_poly, p1, args=(df.Age_months,df.KM,df.Prix), maxfev=500000)
    # ---- Exponential fit
        p0=[param_poly[0],-1e-3,1,1,-1e-2, -1e-6]                           # First parameters 
        param_exp, ier = leastsq(err, p0, args=(df.Age_months,df.KM,df.Prix), maxfev=500000)
        p2=[param_poly[0],-1e-3,1,1,-1e-2, -1e-6]                           # First parameters 
        pfit, pcov, infodict, errmsg, success = leastsq(err, p2, args=(df.Age_months,df.KM,\
                                                               df.Prix), full_output=1, epsfcn=0.000001)
        pfit_results, pcov, infodict, errmsg, success = leastsq(err, pfit, args=(df.Age_months,df.KM,\
                                                                 df.Prix), full_output=1, epsfcn=0.0000001)
        
        # ---- Compute Daniel's Data ------------------------------------------------------------------------------------
        df_Daniel =df_test[(df_test['Modele']==mpl[0]) &(df_test['Puissance']==mpl[1])\
               & (df_test['Litre']==mpl[2])]
        #print(len(df_Daniel))
        # ---- Compute different ages to compute the cote and the VR ----------------------------------------------------
        age_vr = [parse(i) for i in df_Daniel['Date_VR']]
        age_today=[parse(i) for i in df_Daniel['data_month']]
        age_mec =[datetime(int(df_Daniel['Annee'].iat[i]),int(df_Daniel['Mois'].iat[i]),int(df_Daniel['Jour'].iat[i]))\
                   for i in range(len(age_vr))]
        age_month= [round((age_today[i]-age_mec[i]).days/30) for i in range(len(age_vr))]
        df_Daniel['Age_months']=age_month
        
        date_age=[round((age_vr[i]-age_today[i]).days/30) for i in range(len(age_vr))]
        result = [float(i)+float(j) for i,j in zip(date_age,age_month)]
        df_Daniel['Age_VR'] = result 
        
        # ---- Predict the data ----------------------------------------------------------------------------------------
        
        y_new = func_exp2(pfit_results, df_Daniel['Age_months'],df_Daniel.KM.astype(int))   # values of the price with the model
        y_new2 = func_exp2(pfit_results, df_Daniel['Age_VR'],df_Daniel.KM_VR.astype(int))
        y_new=[round(i) for i in y_new]
        y_new2=[round(i) for i in y_new2]
        
        df_Daniel['Cote_Today_C']=y_new
        df_Daniel['Cote_VR_C']=y_new2
        t.append(df_Daniel)
        print(len(t))

  Modele Puissance Litre
0   5008       150  2.00
1
      Modele Puissance Litre
1  EXPERT VU       125  2.00
2
      Modele Puissance Litre
2  EXPERT VU        90  1.60
3
  Modele Puissance Litre
3    208        92  1.60
4
  Modele Puissance Litre
4   2008        92  1.60
5
  Modele Puissance Litre
6    508       115  1.60
6
  Modele Puissance Litre
8    308       115  1.60
7
  Modele Puissance Litre
9    308       120  1.60
8
   Modele Puissance Litre
11   3008       115  1.60
9
   Modele Puissance Litre
12    208        68  1.40


/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:149: RuntimeWarning: overflow encountered in exp


10
   Modele Puissance Litre
13    308        92  1.60
11
   Modele Puissance Litre
14   3008       163  2.00
12
        Modele Puissance Litre
21  PARTNER VU        75  1.60
13
   Modele Puissance Litre
23   3008       120  1.60
14
      Modele Puissance Litre
24  BOXER VU       110  2.20
15
   Modele Puissance Litre
31    108        82  1.20
16
        Modele Puissance Litre
33  PARTNER VU        90  1.60
17
   Modele Puissance Litre
39    208       100  1.60
18
   Modele Puissance Litre
41    508       160  2.00
19
   Modele Puissance Litre
45    308       110  1.20
20
   Modele Puissance Litre
46   5008       120  1.60
21
          Modele Puissance Litre
51  EXPERT TEPEE       125  2.00
22
   Modele Puissance Litre
55    508       150  2.00
23
   Modele Puissance Litre
64    508       163  2.00
24
   Modele Puissance Litre
66    308       130  1.20
25
   Modele Puissance Litre
76    207        70  1.40
26
      Modele Puissance Litre
79  BOXER VU       100  2.20
27
   Modele Puissa

In [22]:
print(len(t))

144


In [23]:
df = pd.concat(t, axis=0)
print(len(df))

2115


In [24]:
df.to_csv('Compa_cote_vr_2017_MPL.csv',sep=';',index=False)